# <img style="float: left; padding-right: 10px; width: 45px" src="https://raw.githubusercontent.com/Harvard-IACS/2018-CS109A/master/content/styles/iacs.png"> CS-S109A Introduction to Data Science 


## Lab 1: Pandas and Web Scraping with Beautiful Soup

**Harvard University**<br>
**Summer 2020**<br>
**Instructor:** Kevin Rader <br>
**Authors:** Rahul Dave, David Sondak, Will Claybaugh, Pavlos Protopapas, Chris Tanner, Eleni Kaxiras, and Kevin Rader

---



In [ ]:
## RUN THIS CELL TO GET THE RIGHT FORMATTING 
import requests
from IPython.core.display import HTML
styles = requests.get("https://raw.githubusercontent.com/Harvard-IACS/2018-CS109A/master/content/styles/cs109.css").text
HTML(styles)

# Table of Contents 
<ol start="0">
<li> Learning Goals </li>
<li> Loading and Cleaning with Pandas</li>
<li> Parsing and Completing the Dataframe  </li>
<li> Grouping </li>
<li> Introduction to Web Servers and HTTP </li>
<li> Download webpages and get basic properties </li>
<li> Parse the page with Beautiful Soup</li>
<li> String formatting</li>
<li> Additonal Python/Homework Comment</li>
<li> Walkthrough Example</li>
</ol>

## Learning Goals

About 6,000 odd "best books" were fetched and parsed from [Goodreads](https://www.goodreads.com). The "bestness" of these books came from a proprietary formula used by Goodreads and published as a list on their web site.

We parsed the page for each book and saved data from all these pages in a tabular format as a CSV file. In this lab we'll clean and further parse the data.  We'll then do some exploratory data analysis to answer questions about these best books and popular genres.  

We will then go back to Goodread's and scrape their Best Books list:

https://www.goodreads.com/list/show/1.Best_Books_Ever?page=1 .

We'll walk through scraping the list pages for the book names/urls. First, we start with an even simpler example.


By the end of this lab, you should be able to:

- Load and systematically address missing values, ancoded as `NaN` values in our data set, for example, by removing observations associated with these values.
- Parse columns in the dataframe to create new dataframe columns.
- Use groupby to aggregate data on a particular feature column, such as author.
- Understand the structure of a web page
- Understand how to use Beautiful soup to scrape content from web pages.
- Feel comfortable storing and manipulating the content in various formats.
- Understand how to convert structured format into a Pandas DataFrame

*This lab corresponds to lectures #1, #2, and maps on to homework #1 and further.*


### Basic EDA workflow

(From the lecture, repeated here for convenience).

The basic workflow is as follows:

1. **Build** a DataFrame from the data (ideally, put all data in this object)
2. **Clean** the DataFrame. It should have the following properties:
    - Each row describes a single object
    - Each column describes a property of that object
    - Columns are numeric whenever appropriate
    - Columns contain atomic properties that cannot be further decomposed
3. Explore **global properties**. Use histograms, scatter plots, and aggregation functions to summarize the data.
4. Explore **group properties**. Use groupby and small multiples to compare subsets of the data.

This process transforms your data into a format which is easier to work with, gives you a basic overview of the data's properties, and likely generates several questions for you to followup in subsequent analysis.

## Part 1: Loading and Cleaning with Pandas 
Read in the `goodreads.csv` file, examine the data, and do any necessary data cleaning. 

Here is a description of the columns (in order) present in this csv file:

```
rating: the average rating on a 1-5 scale achieved by the book
review_count: the number of Goodreads users who reviewed this book
isbn: the ISBN code for the book
booktype: an internal Goodreads identifier for the book
author_url: the Goodreads (relative) URL for the author of the book
year: the year the book was published
genre_urls: a string with '|' separated relative URLS of Goodreads genre pages
dir: a directory identifier internal to the scraping code
rating_count: the number of ratings for this book (this is different from the number of reviews)
name: the name of the book
```

Let us see what issues we find with the data and resolve them.  



----




After loading appropriate libraries


In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import time
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)

### Cleaning: Reading in the data
We read in and clean the data from `goodreads.csv`.

In [ ]:
#Read the data into a dataframe
df = pd.read_csv("../data/goodreads.csv", encoding='utf-8')

#Examine the first few rows of the dataframe
df

Oh no! That does not quite seem to be right. We are missing the column names. We need to add these in! But what are they?

Here is a list of them in order:

`["rating", 'review_count', 'isbn', 'booktype','author_url', 'year', 'genre_urls', 'dir','rating_count', 'name']`


<div class="exercise"><b>Q1.1</b></div>
Use these to load the dataframe properly! And then "head" the dataframe... (you will need to look at the read_csv docs)

In [ ]:
######
# your code here
######


# Examine the first few rows of the dataframe
# df.head()

### Cleaning: Examing the dataframe - quick checks

We should examine the dataframe to get a overall sense of the content. 

<div class="exercise"><b>Q1.2</b></div>
Lets check the types of the columns. What do you find?

In [ ]:
######
# your code here
######



*your answer here*

Notice that `review_count` and `rating_counts` are objects instead of ints, and the `year` is a float!

There are a couple more quick sanity checks to perform on the dataframe. 

In [ ]:
print(df.shape)
df.columns

In [ ]:
df.describe()

### Cleaning: Examining the dataframe - a deeper look

Beyond performing checking some quick general properties of the data frame and looking at the first $n$ rows, we can dig a bit deeper into the values being stored. If you haven't already, check to see if there are any missing values in the data frame.

Let's see for a column which seemed OK to us.


<div class="exercise"><b>Q1.3</b></div>

Use a combination of `np.sum` and `df.isnull()` to determine where missingness occurs


In [ ]:
######
# your code here
######



In [ ]:
#Try to locate where the missing values occur in rating:
df[df.rating.isnull()]

How does `pandas` or `numpy` handle missing values when we try to compute with data sets that include them?

We'll now check if any of the other suspicious columns have missing values.  Let's look at `year` and `review_count` first.

One thing you can do is to try and convert to the type you expect the column to be. If something goes wrong, it likely means your data are bad.

Lets test for missing data:

In [ ]:
df[df.year.isnull()]

df.year.isnull()
df.shape

### Cleaning: Dealing with Missing Values
How should we interpret 'missing' or 'invalid' values in the data (hint: look at where these values occur)? One approach is to simply exclude them from the dataframe. Is this appropriate for all 'missing' or 'invalid' values? 

In [ ]:
# Remove the missing or invalid values in your dataframe

####### 
# your code here
####### 


Ok so we have done some cleaning. What do things look like now? Notice the float has not yet changed.

In [ ]:
df.dtypes

In [ ]:
print(np.sum(df.year.isnull()))
df.shape # We removed seven rows

<div class="exercise"><b>Q1.4</b></div>

Ok so lets fix those types. Convert them to ints. If the type conversion fails, we now know we have further problems.
Hint: use `pd.astype()`.

In [ ]:
######
# your code here
######


Once you do this, we seem to be good on these columns (no errors in conversion). Lets look:

In [ ]:
df.dtypes

Sweet!

Some of the other colums that should be strings have NaN.

In [ ]:
df.loc[df.genre_urls.isnull(), 'genre_urls']=""
df.loc[df.isbn.isnull(), 'isbn']=""

##  Part 2: Parsing and Completing the Data Frame 

We will parse the `author` column from the author_url and `genres` column from the genre_urls. Keep the `genres` column as a string separated by '|'.

We will use panda's `map` to assign new columns to the dataframe.  

Examine an example `author_url` and reason about which sequence of string operations must be performed in order to isolate the author's name.

In [ ]:
#Get the first author_url
test_string = df.author_url[0]
test_string

In [ ]:
#Test out some string operations to isolate the author name

test_string.split('/')[-1].split('.')[1:][0]

<div class="exercise"><b>Q2.1</b></div>

Lets wrap the above code into a function which we will then use

In [ ]:
# Write a function that accepts an author url and returns the author's name based on your experimentation above


def get_author(url):
    ####### 
    # your code here
    ####### 
   
    return name

In [ ]:
# Apply the get_author function to the 'author_url' column using '.map' 
# and add a new column 'author' to store the names

df['author'] = df.author_url.map(get_author)
df.author[0:5]


Now let's parse out the genres from `genre_url`.  

This is a little more complicated because there be more than one genre.


In [ ]:
#Examine some examples of genre_urls
df.genre_urls.head()

In [ ]:
######
#your code here
######


#Test out some string operations to isolate the genre name
test_genre_string=df.genre_urls[0]


genres=test_genre_string.strip().split('|')
for e in genres:
    print(e.split('/')[-1])
    "|".join(genres)

<div class="exercise"><b>Q2.2</b></div>

Write a function that accepts a genre url and returns the genre name based on your experimentation above



In [ ]:
def split_and_join_genres(url):
    ######
    # your code here
    ######
    genres=url.strip().split('|')
    genres=[e.split('/')[-1] for e in genres]
    return "|".join(genres)

Test your function

In [ ]:
split_and_join_genres("/genres/young-adult|/genres/science-fiction")

In [ ]:
split_and_join_genres("")

<div class="exercise"><b>Q2.3</b></div>

Use map again to create a new "genres" column

In [ ]:

df['genres']=df.genre_urls.map(split_and_join_genres)
df.head()

Finally, let's pick an author at random so we can see the results of the transformations.  Scroll to see the `author` and `genre` columns that we added to the dataframe.

In [ ]:
df[df.author == "Marguerite_Yourcenar"]

Let us delete the `genre_urls` column.

In [ ]:
del df['genre_urls']

And then save the dataframe out!

In [ ]:
df.to_csv("../data/cleaned-goodreads.csv", index=False, header=True)

---

## Part 3: Grouping 

<div class="exercise"><b>Q3.1</b></div>

It appears that some books were written in negative years!  Print out the observations that correspond to negative years.  What do you notice about these books?  

In [ ]:
######
# your code here
######


# Note: these are books written before the Common Era (BCE, equivalent to BC).

We could determine the "best book" by year! For this we use Panda's `groupby()`. `groupby()` allows grouping a dataframe by any (usually categorical) variable. Would it make sense to ever groupby integer variables? Floating point variables?

But more useful, let's consider grouping by author:

In [ ]:
df

In [ ]:
dfgb_author = df.groupby('author')
type(dfgb_author)

Perhaps we want the number of books each author wrote

In [ ]:
dfgb_author.count().head()

Lots of useless info there. One column should suffice.  But perhaps you want more detailed info...

In [ ]:
dfgb_author[['rating', 'rating_count', 'review_count', 'year']].describe()

You can also access a `groupby` dictionary style.

In [ ]:
ratingdict = {}
for author, subset in dfgb_author:
    ratingdict[author] = (subset['rating'].mean(), subset['rating'].std())
#ratingdict

<div class="exercise"><b>Q3.2</b></div>

This is a longer exercise, and you may want to use several cells to answer the following:"

- Group the dataframe by `author`. Include the following columns: `rating`, `name`, `author`. For the aggregation of the `name` column which includes the names of the books create a list with the strings containing the name of each book. Make sure that the way you aggregate the rest of the columns make sense! 

- Create a new column with number of books for each author and find the most prolific author!

In [ ]:
###### Before we start : what do we do about these titles where 'name' is unreadable? Try different encodings?
auth_name = 'A_id_al_Qarni'
df[df.author == auth_name].head()

In [ ]:
df[df.author == auth_name].iat[0,8].encode('UTF-16')

In [ ]:
# let's examine the columns we have
df.columns

Create the GroupBy table

In [ ]:
authors = df.copy()
authors = authors[['rating','name','author']].groupby('author').agg(
    ######
    # your code here
    # Hint1: calculate the mean of 'rating' across authors and
    # Hint2: .join the 'name' (of books) into a single string with | as the separator
    ######
    { }
)

In [ ]:
authors = authors.reset_index()
authors.head()

In [ ]:
# replace 'name' with a list of book names after using str.split
######
# your code here
######

#authors.head()

In [ ]:
# count the books - create new column
len(authors.name[0])

In [ ]:
# count the number of books for each author 

######
# your code here
######


In [ ]:
# determine who has been the most prolific (in this list)

######
# your code here
######


Wow! **Stephen King** wins with **56 books**!

<div class="exercise"><b>BONUS QUESTION</b></div>

Lets get the best-rated book(s) for every year in our Goodreads dataframe.

In [ ]:

######
# your code here
# Hint: don't try to slice, rather use a for loop to iterate through each  
# 'group' within the groupBy object
######




---

# Introduction to Web Servers and HTTP

A web server is just a computer -- usually a powerful one, but ultimately it's just another computer -- that runs a long/continuous process that listens for requests on a pre-specified (Internet) _port_ on your computer. It responds to those requests via a protocol called HTTP (HyperText Transfer Protocol). HTTPS is the secure version. When we use a web browser and navigate to a web page, our browser is actually sending a request on our behalf to a specific web server. The browser request is essentially saying "hey, please give me the web page contents", and it's up to the browser to correctly render that raw content into a coherent manner, dependent on the format of the file. For example, HTML is one format, XML is another format, and so on.

Ideally (and usually), the web server complies with the request and all is fine. As part of this communication exchange with web servers, the server also sends a status code.
- If the code starts with a **2**, it means the request was successful.
- If the code starts with a **4**, it means there was a client error (you, as the user, are the client). For example, ever receive a 404 File Not Found error because a web page doesn't exist? This is an example of a client error, because you are requesting a bogus item.
- If the code starts with a **5**, it means there was a server error (often that your request was incorrectly formed).

[Click here](https://www.restapitutorial.com/httpstatuscodes.html) for a full list of status codes.

As an analogy, you can think of a web server as being like a server at a restaurant; its goal is _serve_ you your requests. When you try to order something not on the menu (i.e., ask for a web page at a wrong location), the server says 'sorry, we don't have that' (i.e., 404, client error; your mistake).

**IMPORTANT:**
As humans, we visit pages in a sane, reasonable rate. However, as we start to scrape web pages with our computers, we will be sending requests with our code, and thus, we can make requests at an incredible rate. This is potentially dangerous because it's akin to going to a restaurant and bombarding the server(s) with thousands of food orders. Very often, the restaurant will ban you (i.e., Harvard's network gets banned from the website, and you are potentially held responsible in some capacity?). It is imperative to be responsible and careful. In fact, this act of flooding web pages with requests is the single-most popular, yet archiac, method for maliciously attacking websites / computers with Internet connections. In short, be respectful and careful with your decisions and code. It is better to err on the side of caution, which includes using the **``time.sleep()`` function** to pause your code's execution between subsequent requests. ``time.sleep(2)`` should be fine when making just a few dozen requests. Each site has its own rules, which are often visible via their site's ``robots.txt`` file.

### Additional Resources

**HTML:** if you are not familiar with HTML see https://www.w3schools.com/html/ or one of the many tutorials on the internet.

**Document Object Model (DOM):** for more on this programming interface for HTML and XML documents see https://www.w3schools.com/js/js_htmldom.asp.

## Part 4: Download webpages and get basic properties

``Requests`` is a highly useful Python library that allows us to fetch web pages.
``BeautifulSoup`` is a phenomenal Python library that allows us to easily parse web content and perform basic extraction.

If one wishes to scrape webpages, one usually uses ``requests`` to fetch the page and ``BeautifulSoup`` to parse the page's meaningful components. Webpages can be messy, despite having a structured format, which is why BeautifulSoup is so handy.

Let's get started:

In [ ]:
from bs4 import BeautifulSoup
import requests

To fetch a webpage's content, we can simply use the ``get()`` function within the requests library:

In [ ]:
url = "https://www.npr.org/2018/11/05/664395755/what-if-the-polls-are-wrong-again-4-scenarios-for-what-might-happen-in-the-elect"
response = requests.get(url) # you can use any URL that you wish

The response variable has many highly useful attributes, such as:
- status_code
- text
- content

Let's try each of them!

### response.status_code

In [ ]:
response.status_code

You should have received a status code of 200, which means the page was successfully found on the server and sent to receiver (aka client/user/you). [Again, you can click here](https://www.restapitutorial.com/httpstatuscodes.html) for a full list of status codes.

### response.text


In [ ]:
response.text

Holy moly! That looks awful. If we use our browser to visit the URL, then right-click the page and click 'View Page Source', we see that it is identical to this chunk of glorious text.

### response.content

In [ ]:
response.content

What?! This seems identical to the ``.text`` field. However, the careful eye would notice that the very 1st characters differ; that is, ``.content`` has a *b'* character at the beginning, which in Python syntax denotes that the data type is bytes, whereas the ``.text`` field did not have it and is a regular String.

Ok, so that's great, but how do we make sense of this text? We could manually parse it, but that's tedious and difficult. As mentioned, BeautifulSoup is specifically designed to parse this exact content (any webpage content).

## BEAUTIFUL SOUP
![title](../images/soup_for_you.jpg) (property of NBC)


The [documentation for BeautifulSoup is found here](https://www.crummy.com/software/BeautifulSoup/bs4/doc/).

A BeautifulSoup object can be initialized with the ``.content`` from request and a flag denoting the type of parser that we should use. For example, we could specify ``html.parser``, ``lxml``, etc [documentation here](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#differences-between-parsers). Since we are interested in standard webpages that use HTML, let's specify the html.parser:

In [ ]:
soup = BeautifulSoup(response.content, "html.parser")
soup

Alright! That looks a little better; there's some whitespace formatting, adding some structure to our content! HTML code is structured by `<tags>`. Every tag has an opening and closing portion, denoted by ``< >`` and ``</ >``, respectively. If we want just the text (not the tags), we can use:

In [ ]:
soup.get_text()

There's some tricky Javascript still nesting within it, but it definitely cleaned up a bit. On other websites, you may find even clearer text extraction.

As detailed in the [BeautifulSoup documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/), the easiest way to navigate through the tags is to simply name the tag you're interested in. For example:

In [ ]:
soup.head # fetches the head tag, which ecompasses the title tag

Usually head tags are small and only contain the most important contents; however, here, there's some Javascript code. The ``title`` tag resides within the head tag.

In [ ]:
soup.title # we can specifically call for the title tag

This result includes the tag itself. To get just the text within the tags, we can use the ``.name`` property.

In [ ]:
soup.title.string

We can navigate to the parent tag (the tag that encompasses the current tag) via the ``.parent`` attribute:

In [ ]:
soup.title.parent.name

## Parse the page with Beautiful Soup
In HTML code, paragraphs are often denoated with a ``<p>`` tag.

In [ ]:
soup.p

This returns the first paragraph, and we can access properties of the given tag with the same syntax we use for dictionaries and dataframes:

In [ ]:
soup.p['class']

In addition to 'paragraph' (aka p) tags, link tags are also very common and are denoted by ``<a>`` tags

In [ ]:
soup.a

It is called the a tag because links are also called 'anchors'. Nearly every page has multiple paragraphs and anchors, so how do we access the subsequent tags? There are two common functions, `.find()` and `.find_all()`.

In [ ]:
soup.find('title')

In [ ]:
soup.find_all('title')

Here, the results were seemingly the same, since there is only one title to a webpage. However, you'll notice that ``.find_all()`` returned a list, not a single item. Sure, there was only one item in the list, but it returned a list. As the name implies, find_all() returns all items that match the passed-in tag.

In [ ]:
soup.find_all('a')

Look at all of those links! Amazing. It might be hard to read but the **href** portion of an *a* tag denotes the URL, and we can capture it via the ``.get()`` function.

In [ ]:
for link in soup.find_all('a'): # we could optionally pass the href=True flag .find_all('a', href=True)
    print(link.get('href'))

Many of those links are relative to the current URL (e.g., /section/news/).

In [ ]:
paragraphs = soup.find_all('p')
paragraphs

If we want just the paragraph text:

In [ ]:
for pa in paragraphs:
    print(pa.get_text())

Since there are multiple tags and various attributes, it is useful to check the data type of BeautifulSoup objects:

In [ ]:
type(soup.find('p'))

Since the ``.find()`` function returns a BeautifulSoup element, we can tack on multiple calls that continue to return elements:

In [ ]:
soup.find('p')

In [ ]:
soup.find('p').find('a')

In [ ]:
soup.find('p').find('a').attrs['href'] # att

In [ ]:
soup.find('p').find('a').text

That doesn't look pretty, but it makes sense because if you look at what ``.find('a')`` returned, there is plenty of whitespace. We can remove that with Python's built-in ``.strip()`` function.

In [ ]:
soup.find('p').find('a').text.strip()

**NOTE:** above, we accessed the attributes of a link by using the property ``.attrs``. ``.attrs`` takes a dictionary as a parameter, and in the example above, we only provided the _key_, not a _value_, too. That is, we only cared that the ``<a>`` tag had an attribute named ``href`` (which we grabbed by typing that command), and we made no specific demands on what the value must be. In other words, regardless of the value of _href_, we grabbed that element. Alternatively, if you inspect your HTML code and notice select regions for which you'd like to extract text, you can specify it as part of the attributes, too!

For example, in the full ``response.text``, we see the following line:

``<header class="npr-header" id="globalheader" aria-label="NPR header">``

Let's say that we know that the information we care about is within tags that match this template (i.e., **class** is an attribute, and its value is **'npr-header'**).

In [ ]:
soup.find('header', attrs={'class':'npr-header'})

This matched it! We could then continue further processing by tacking on other commands:

In [ ]:
soup.find('header', attrs={'class':'npr-header'}).find_all("li") # li stands for list items

This returns all of our list items, and since it's within a particular header section of the page, it appears they are links to menu items for navigating the webpage. If we wanted to grab just the links within these:

In [ ]:
menu_links = set()
for list_item in soup.find('header', attrs={'class':'npr-header'}).find_all("li"):
    for link in list_item.find_all('a', href=True):
        menu_links.add(link)
menu_links # a unique set of all the seemingly important links in the header

## TAKEAWAY LESSON
The above tutorial isn't meant to be a study guide to memorize; its point is to show you the most important functionaity that exist within BeautifulSoup, and to illustrate how one can access different pieces of content. No two web scraping tasks are identical, so it's useful to play around with code and try different things, while using the above as examples of how you may navigate between different tags and properties of a page. Don't worry; we are always here to help when you get stuck!

# String formatting
As we parse webpages, we may often want to further adjust and format the text to a certain way.

For example, say we wanted to scrape a polical website that lists all US Senators' name and office phone number. We may want to store information for each senator in a dictionary. All senators' information may be stored in a list. Thus, we'd have a list of dictionaries. Below, we will initialize such a list of dictionary (it has only 3 senators, for illustrative purposes, but imagine it contains many more).

In [ ]:
# this is a bit clumsy of an initialization, but we spell it out this way for clarity purposes
# NOTE: imagine the dictionary were constructed in a more organic manner
senator1 = {"name":"Lamar Alexander", "number":"555-229-2812"}
senator2 = {"name":"Tammy Baldwin", "number":"555-922-8393"}
senator3 = {"name":"John Barrasso", "number":"555-827-2281"}
senators = [senator1, senator2, senator3]
print(senators)

In the real-world, we may not want the final form of our information to be in a Python dictionary; rather, we may need to send an email to people in our mailing list, urging them to call their senators. If we have a templated format in mind, we can do the following:

In [ ]:
email_template = """Please call {name} at {number}"""
for senator in senators:
    print(email_template.format(**senator))

**Please [visit here](https://docs.python.org/3/library/stdtypes.html#str.format)** for further documentation
                      
Alternatively, one can also format their text via the ``f'-strings`` property. [See documentation here](https://docs.python.org/3/reference/lexical_analysis.html#f-strings). For example, using the above data structure and goal, one could yield identical results via:

In [ ]:
for senator in senators:
    print(f"Please call {senator['name']} at {senator['number']}")

Additionally, sometimes we wish to search large strings of text. If we wish to find all occurrences within a given string, a very mechanical, procedural way of doing it would be to use the ``.find()`` function in Python and to repeatedly update the starting index from which we are looking.

## Regular Expressions
A way more suitable and powerful way is to use Regular Expressions, which is a pattern matching mechanism used throughout Computer Science and programming (it's not just specific to Python). A tutorial on Regular Expressions (aka regex) is beond this lab, but below are many great resources that we recommend, if you are interested in them (could be very useful for a homework problem):
- https://docs.python.org/3.3/library/re.html
- https://regexone.com
- https://docs.python.org/3/howto/regex.html.

# Additonal Python/Homework Comment
In Homework #1, we ask you to complete functions that have signatures with a syntax you may not have seen before:

``def create_star_table(starlist: list) -> list:``

To be clear, this syntax merely means that the input parameter must be a list, and the output must be a list. It's no different than any other function, it just puts a requirement on the behavior of the function.

It is **typing** our function. Please [see this documention if you have more questions.](https://docs.python.org/3/library/typing.html)

# Walkthrough Example (of Web Scraping)
We're going to see the structure of Goodread's best books list online. We'll use the Developer tools in chrome, safari and firefox have similar tools available. To get this page we use the `requests` module. But first we should check if the company's policy allows scraping. Check the [robots.txt](https://www.goodreads.com/robots.txt) to find what sites/elements are not accessible. Please read and verify.

![](images/goodreads1.png)

In [ ]:
url="https://www.npr.org/2018/11/05/664395755/what-if-the-polls-are-wrong-again-4-scenarios-for-what-might-happen-in-the-elect"
response = requests.get(url)
# response.status_code
# response.content

# Beautiful Soup (library) time!
soup = BeautifulSoup(response.content, "html.parser")
    #print(soup)
    # soup.prettify()
soup.find("title")

    # Q1: how do we get the title's text?

    # Q2: how do we get the webpage's entire content?

In [ ]:
URLSTART="https://www.goodreads.com"
BESTBOOKS="/list/show/1.Best_Books_Ever?page="
url = URLSTART+BESTBOOKS+'1'
print(url)
page = requests.get(url)

We can see properties of the page. Most relevant are `status_code` and `text`. The former tells us  if the web-page was found, and if found , ok. (See lecture notes.)

In [ ]:
page.status_code # 200 is good

In [ ]:
page.text[:5000]

Let us write a loop to fetch 2 pages of "best-books" from goodreads. Notice the use of a format string. This is an example of old-style python format strings

In [ ]:
URLSTART="https://www.goodreads.com"
BESTBOOKS="/list/show/1.Best_Books_Ever?page="
for i in range(1,3):
    bookpage=str(i)
    stuff=requests.get(URLSTART+BESTBOOKS+bookpage)
    filetowrite="../data/page"+ '%02d' % i + ".html"
    print("FTW", filetowrite)
    fd=open(filetowrite,"w")
    fd.write(stuff.text)
    fd.close()
    time.sleep(2)

## 2. Parse the page, extract book urls

Notice how we do file input-output, and use beautiful soup in the code below. The `with` construct ensures that the file being read is closed, something we do explicitly for the file being written. We look for the elements with class `bookTitle`, extract the urls, and write them into a file

In [ ]:
bookdict={}
for i in range(1,3):
    books=[]
    stri = '%02d' % i
    filetoread="../data/page"+ stri + '.html'
    print("FTW", filetoread)
    with open(filetoread) as fdr:
        data = fdr.read()
    soup = BeautifulSoup(data, 'html.parser')
    for e in soup.select('.bookTitle'):
        books.append(e['href'])
    print(books[:10])
    bookdict[stri]=books
    fd=open("../data/list"+stri+".txt","w")
    fd.write("\n".join(books))
    fd.close()

Here is one of everyone's favorite books from HS American Lit. class:

In [ ]:
bookdict['02'][0]

 Lets go look at the first URLs on both pages

![](images/goodreads2.png)

## 3. Parse a book page, extract book properties

Ok so now lets dive in and get one of these these files and parse them.

In [ ]:
furl=URLSTART+bookdict['02'][0]
furl

![](images/goodreads3.png)

In [ ]:
fstuff=requests.get(furl)
print(fstuff.status_code)

In [ ]:
#d=BeautifulSoup(fstuff.text, 'html.parser')
# try this to take care of arabic strings
d = BeautifulSoup(fstuff.text, 'html.parser')

In [ ]:
d.select("meta[property='og:title']")[0]['content']

Lets get everything we want...

In [ ]:
#d=BeautifulSoup(fstuff.text, 'html.parser', from_encoding="utf-8")
print(
"title:", d.select_one("meta[property='og:title']")['content'],"\n",
"isbn:", d.select("meta[property='books:isbn']")[0]['content'],"\n",
"type:", d.select("meta[property='og:type']")[0]['content'],"\n",
"author:", d.select("meta[property='books:author']")[0]['content'],"\n",
#"average rating", d.select_one("span.average").text,"\n",
"ratingCount:", d.select("meta[itemprop='ratingCount']")[0]["content"],"\n"
#"reviewCount", d.select_one("span.count")["title"]
)

Ok, now that we know what to do, lets wrap our fetching into a proper script. So that we dont overwhelm their servers, we will only fetch 5 from each page, but you get the idea...

We'll segue of a bit to explore new style format strings. See https://pyformat.info for more info.

In [ ]:
"list{:0>2}.txt".format(3)

In [ ]:
a = "4"
b = 4
class Four:
    def __str__(self):
        return "Fourteen"
c=Four()

In [ ]:
"The hazy cat jumped over the {} and {} and {}".format(a, b, c)

## 4. Set up a pipeline for fetching and parsing

Ok lets get back to the fetching...

In [ ]:
fetched=[]
for i in range(1,3):
    with open("../data/list{:0>2}.txt".format(i)) as fd:
        counter=0
        for bookurl_line in fd:
            if counter > 4:
                break
            bookurl=bookurl_line.strip()
            stuff=requests.get(URLSTART+bookurl)
            filetowrite=bookurl.split('/')[-1]
            filetowrite="../data/"+str(i)+"_"+filetowrite+".html"
            print("FTW", filetowrite)
            fd=open(filetowrite,"w")
            fd.write(stuff.text)
            fd.close()
            fetched.append(filetowrite)
            time.sleep(2)
            counter=counter+1
            
print(fetched)

Ok we are off to parse each one of the html pages we fetched. We have provided the skeleton of the code and the code to parse the year, since it is a bit more complex...see the difference in the screenshots above. 

In [ ]:
import re
yearre = r'\d{4}'
def get_year(d):
    if d.select_one("nobr.greyText"):
        return d.select_one("nobr.greyText").text.strip().split()[-1][:-1]
    else:
        thetext=d.select("div#details div.row")[1].text.strip()
        rowmatch=re.findall(yearre, thetext)
        if len(rowmatch) > 0:
            rowtext=rowmatch[0].strip()
        else:
            rowtext="NA"
        return rowtext

<div class="exercise"><b>Q4.1</b></div>

Your job is to fill in the code to get the genres.

In [ ]:
def get_genres(d):
    ######
    # your code here
    ######


In [ ]:

listofdicts=[]
for filetoread in fetched:
    print(filetoread)
    td={}
    with open(filetoread) as fd:
        datext = fd.read()
    d=BeautifulSoup(datext, 'html.parser')
    td['title']=d.select_one("meta[property='og:title']")['content']
    td['isbn']=d.select_one("meta[property='books:isbn']")['content']
    td['booktype']=d.select_one("meta[property='og:type']")['content']
    td['author']=d.select_one("meta[property='books:author']")['content']
    #td['rating']=d.select_one("span.average").text
    td['year'] = get_year(d)
    td['file']=filetoread
    glist = get_genres(d)
    td['genres']="|".join(glist)
    listofdicts.append(td)

In [ ]:
listofdicts[0]

Finally lets write all this stuff into a csv file which we will use to do analysis.

In [ ]:
df2 = pd.DataFrame.from_records(listofdicts)
df2

In [ ]:
# and save as a csv
df2.to_csv("../data/meta_utf8_EK.csv", index=False, header=True)